In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import csv
import os

import math
from multiprocessing import Pool, cpu_count
from functools import partial

In [3]:
print("Acquiring datasets\n")

train_query          = np.load("../DatasetCreation/training_dataset.npy")
train_support        = np.load("../DatasetCreation/training_support_dataset.npy")
train_query_neighbors= np.load("../DatasetCreation/train_query_neighbors.npy")

val_query            = np.load("../DatasetCreation/validation_dataset.npy")
val_support          = np.load("../DatasetCreation/validation_support_dataset.npy")
val_query_neighbors  = np.load("../DatasetCreation/val_query_neighbors.npy")

print("Done. neighbor_array shape:", train_query_neighbors.shape)

class TransformerEmbed(nn.Module):
    def __init__(self, input_dim=1, d_model=64, nhead=4, num_layers=2):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=128,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # Remove the pooling layer to maintain temporal resolution
        # Instead add a final projection to reduce channel dimension if needed
        self.final_proj = nn.Linear(d_model, 32)  # projects each timestep to 32 dimensions
    
    def forward(self, x):
        # x: (batch, seq_len=192, 1)
        x = self.input_proj(x)              # => (batch, 192, d_model)
        x = self.transformer_encoder(x)      # => (batch, 192, d_model)
        x = self.final_proj(x)              # => (batch, 192, 32)
        return x

class SiameseTransformer(nn.Module):
    def __init__(self, transformer_model):
        super().__init__()
        self.transformer = transformer_model
        
        # Add comparison module to generate richer difference features
        self.comparison = nn.Sequential(
            nn.Linear(64, 32),  # 64 = concatenated features per timestep
            nn.ReLU(),
            nn.Linear(32, 32)
        )
    
    def forward(self, x1, x2):
        emb1 = self.transformer(x1)  # => (B, 192, 32)
        emb2 = self.transformer(x2)  # => (B, 192, 32)
        
        # Concatenate embeddings along feature dimension
        concat_features = torch.cat([emb1, emb2], dim=-1)  # => (B, 192, 64)
        
        # Generate rich difference features for each timestep
        diff = self.comparison(concat_features)  # => (B, 192, 32)
        return diff

class LSTMForecaster(nn.Module):
    def __init__(self, input_dim=1, diff_dim=32, hidden_dim=256, num_layers=3):
        super().__init__()
        
        # Modified to handle sequence of difference vectors
        self.diff_encoder = nn.LSTM(
            input_size=diff_dim,
            hidden_size=hidden_dim,
            num_layers=1,
            batch_first=True
        )
        
        # Main LSTM now takes concatenated input
        self.lstm = nn.LSTM(
            input_size=input_dim + hidden_dim,  # Concatenated input and encoded diff
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        
        self.fc_out = nn.Linear(hidden_dim, 1)

    def forward(self, support_future, diff_vec):
        """
        support_future: (B, 192, 1)
        diff_vec: (B, 192, 32) - now a sequence of difference vectors
        => returns (B, 192, 1)
        """
        # Encode the sequence of difference vectors
        diff_encoded, (h0, c0) = self.diff_encoder(diff_vec)  # => (B, 192, hidden_dim)
        
        # Concatenate encoded differences with support_future at each timestep
        combined_input = torch.cat([support_future, diff_encoded], dim=-1)  # => (B, 192, input_dim + hidden_dim)
        
        # Process through main LSTM
        lstm_out, _ = self.lstm(combined_input)  # => (B, 192, hidden_dim)
        pred = self.fc_out(lstm_out)             # => (B, 192, 1)
        
        return pred

def get_batch(query_dataset, support_dataset, neighbor_array, batch_size):
    """
    neighbor_array: shape (NQ,). neighbor_array[i] gives the best support index for query i
    query_dataset, support_dataset: shape (NQ,384) or (NS,384)
    
    returns x_test, y_test, x_support, y_support as Tensors
    """
    idxs = np.random.choice(len(query_dataset), batch_size, replace=False)

    x_test_list = []
    y_test_list = []
    x_support_list = []
    y_support_list = []

    for idx in idxs:
        chunk = query_dataset[idx]       # shape (384,)
        test_past   = chunk[:192]       # shape (192,)
        test_future = chunk[192:]       # shape (192,)

        # get precomputed support index
        idx_support = neighbor_array[idx]
        support_chunk = support_dataset[idx_support]
        support_past   = support_chunk[:192]
        support_future = support_chunk[192:]

        x_test_list.append(test_past)
        y_test_list.append(test_future)
        x_support_list.append(support_past)
        y_support_list.append(support_future)

    # shape => (B,192)
    x_test_arr     = np.array(x_test_list,     dtype=np.float32)
    y_test_arr     = np.array(y_test_list,     dtype=np.float32)
    x_support_arr  = np.array(x_support_list,  dtype=np.float32)
    y_support_arr  = np.array(y_support_list,  dtype=np.float32)

    # reshape => (B,192,1)
    x_test_tensor     = torch.tensor(x_test_arr).unsqueeze(-1)
    y_test_tensor     = torch.tensor(y_test_arr).unsqueeze(-1)
    x_support_tensor  = torch.tensor(x_support_arr).unsqueeze(-1)
    y_support_tensor  = torch.tensor(y_support_arr).unsqueeze(-1)

    return x_test_tensor, y_test_tensor, x_support_tensor, y_support_tensor


Acquiring datasets

Done. neighbor_array shape: (198071,)


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize models
transformer = TransformerEmbed(input_dim=1, d_model=64, nhead=4, num_layers=2)
siamese_model = SiameseTransformer(transformer)
lstm_model = LSTMForecaster(input_dim=1, diff_dim=32, hidden_dim=256, num_layers=3)

# Move models to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
siamese_model = siamese_model.to(device)
lstm_model = lstm_model.to(device)

# Loss functions and optimizer
criterion_mae = nn.L1Loss()
criterion_mse = nn.MSELoss()
# Combine parameters from both models for single optimizer
params = list(siamese_model.parameters()) + list(lstm_model.parameters())
optimizer = torch.optim.Adam(params, lr=0.001)

In [ ]:
# Training constants
EPOCHS = 10000
BATCH_SIZE = 16
STEPS_PER_EPOCH = 100

# Setup CSV logging
csv_filename = 'training_log.csv'
with open(csv_filename, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["epoch", "MAE", "MSE"])

# Training loop
for epoch in range(EPOCHS):
    epoch_mae = 0.0
    epoch_mse = 0.0
    
    for step in range(STEPS_PER_EPOCH):
        # Get batch
        x_test, y_test, x_support, y_support = get_batch(
            query_dataset=train_query,
            support_dataset=train_support,
            neighbor_array=train_query_neighbors,
            batch_size=BATCH_SIZE
        )
        
        # Move to device
        x_test = x_test.to(device)
        y_test = y_test.to(device)
        x_support = x_support.to(device)
        y_support = y_support.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        
        # Get high-resolution difference vectors
        diff_vec = siamese_model(x_test, x_support)  # => (B, 192, 32)
        
        # Generate forecast
        y_pred = lstm_model(y_support, diff_vec)     # => (B, 192, 1)
        
        # Compute losses
        loss_mae = criterion_mae(y_pred, y_test)
        loss_mse = criterion_mse(y_pred, y_test)
        
        # Combined loss for backprop
        loss = loss_mae + 0.5 * loss_mse  # weighting can be adjusted
        
        # Backward pass and optimization
        loss.backward()
        # Optional gradient clipping
        torch.nn.utils.clip_grad_norm_(params, max_norm=1.0)
        optimizer.step()
        
        # Update metrics
        epoch_mae += loss_mae.item()
        epoch_mse += loss_mse.item()
    
    # Compute epoch averages
    avg_mae = epoch_mae / STEPS_PER_EPOCH
    avg_mse = epoch_mse / STEPS_PER_EPOCH
    
    # Print epoch summary
    print(f"Epoch {epoch+1}/{EPOCHS} - MAE: {avg_mae:.4f}, MSE: {avg_mse:.4f}")
    
    # Log to CSV
    with open(csv_filename, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([epoch+1, avg_mae, avg_mse])
    
    # Save checkpoints every N epochs
    if (epoch + 1) % 100 == 0:
        checkpoint = {
            'epoch': epoch + 1,
            'siamese_state_dict': siamese_model.state_dict(),
            'lstm_state_dict': lstm_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'mae': avg_mae,
            'mse': avg_mse
        }
        torch.save(checkpoint, f'checkpoint_epoch_{epoch+1}.pth')

print("Training completed!")

Epoch 1/10000 - MAE: 782.1811, MSE: 977428.5025
Epoch 2/10000 - MAE: 758.7141, MSE: 930934.8784
Epoch 3/10000 - MAE: 747.9802, MSE: 906904.1025
Epoch 4/10000 - MAE: 703.8657, MSE: 843581.5100
Epoch 5/10000 - MAE: 700.0416, MSE: 831013.4894
Epoch 6/10000 - MAE: 658.3416, MSE: 767223.3528
Epoch 7/10000 - MAE: 650.9808, MSE: 755766.0984
Epoch 8/10000 - MAE: 627.2200, MSE: 715999.2659
Epoch 9/10000 - MAE: 615.6869, MSE: 693099.8959
Epoch 10/10000 - MAE: 585.3372, MSE: 644760.7325
Epoch 11/10000 - MAE: 574.7885, MSE: 623367.9153
Epoch 12/10000 - MAE: 561.0992, MSE: 602733.3981
Epoch 13/10000 - MAE: 554.4943, MSE: 587237.3828
Epoch 14/10000 - MAE: 527.5872, MSE: 544356.6859
Epoch 15/10000 - MAE: 524.2096, MSE: 535006.6928
Epoch 16/10000 - MAE: 496.6518, MSE: 502613.4264
Epoch 17/10000 - MAE: 495.7547, MSE: 495423.8000
Epoch 18/10000 - MAE: 462.1170, MSE: 444160.3942
Epoch 19/10000 - MAE: 447.4118, MSE: 419767.6603
Epoch 20/10000 - MAE: 450.1264, MSE: 424104.0656
Epoch 21/10000 - MAE: 431.129